In [11]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key ="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        #Load any messages stored in memory into the prompt
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

#이거 계속 add해서 불러오기 귀찮으니 RunablePassthrough로 만들어서 불러오자
#Load the chat history from memory
#chain.invoke({
#    "chat_history":memory.load_memory_variables({})["chat_history"],
#    "question":"Hello, how are you?"
#})


def load_memory(input):
    #print(input)
    #print("***************")
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context({"input": question},{"output": result.content},)
    print(result)




In [12]:
invoke_chain("My name is Yoon")

content='Nice to meet you, Yoon! How can I assist you today?'


In [13]:
#대화내용이 저장되있는것을 확인할수있는 대답을함
invoke_chain("What is my name?")

content='Your name is Yoon.'
